In [142]:
import numpy as np

%load_ext autoreload
%autoreload 2
import my_baselines

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Purchase prediction

In [94]:
with open('train.json') as f:
    data = f.readlines()

import ast
data = [ast.literal_eval(x) for x in data]
data[0]

({'categories': [['Clothing, Shoes & Jewelry', 'Women'],
   ['Clothing, Shoes & Jewelry',
    'Novelty, Costumes & More',
    'Novelty',
    'Clothing',
    'Women',
    'Leggings']],
  'categoryID': 0,
  'helpful': {'nHelpful': 0, 'outOf': 0},
  'itemID': 'I402344648',
  'rating': 4.0,
  'reviewHash': 'R798569390',
  'reviewText': "The model in this picture has them rolled up at the top because they are actually very high waisted! that's my only complaint though, because they are very good quality, and fit really well! I am 5'2&#34; 120lbs with thick thighs and i love them i can't wait to wear them out!",
  'reviewTime': '09 26, 2013',
  'reviewerID': 'U490934656',
  'summary': 'High Waisted',
  'unixReviewTime': 1380153600},
 200000)

In [95]:
trainX, valX = data[:100000], data[100000:]
len(trainX), len(valX)

(100000, 100000)

In [100]:
reviewer_item_pair = {}
reviewer_list = []
item_list = []

for d in data:
    reviewer = d['reviewerID']
    item = d['itemID']
    pair = reviewer_item_pair.get(reviewer, [])
    pair.append(item)
    reviewer_item_pair[reviewer] = pair
    reviewer_list.append(reviewer)
    item_list.append(item)
    

In [111]:
import random
cnt = 0
neg_pair = []

while(cnt < 100000):
    reviewer = random.choice(reviewer_list)
    item = random.choice(item_list)
    if item not in reviewer_item_pair[reviewer]:
        neg_pair.append((reviewer, item))
        cnt += 1

In [113]:
len(neg_pair), neg_pair[0]

(100000, ('U582106887', 'I145325374'))

In [147]:
##########################
# 2
##########################

In [148]:
##########################
# 3
##########################
reviewer_cat_pair = {}
item_cat_pair = {}

for d in data:
    reviewer = d['reviewerID']
    item = d['itemID']
    cat = d['categories']
    
    reviewer_pair = reviewer_cat_pair.get(reviewer, [])
    item_pair = item_cat_pair.get(item, [])
    for c in cat:
        if c not in reviewer_pair:
            reviewer_pair.append(c)
        if c not in item_pair:
            item_pair.append(c)
        
    reviewer_cat_pair[reviewer] = reviewer_pair
    item_cat_pair[item] = item_pair
    
def p3(u, i):
    try:
        reviewer_pair = reviewer_cat_pair[u]
        item_pair = item_cat_pair[i]
    except KeyError:
        return 0
    
    for i in item_pair:
        if i in reviewer_pair: return 1
    return 0

my_baselines.purchaseBaseline(p3)

In [146]:
##########################
# 4
##########################

# Rating prediction

In [126]:
data[0]['categories'][1]

['Clothing, Shoes & Jewelry',
 'Novelty, Costumes & More',
 'Novelty',
 'Clothing',
 'Women',
 'Leggings']

In [131]:
data[0]['categories'] + data[1]['categories']

[['Clothing, Shoes & Jewelry', 'Women'],
 ['Clothing, Shoes & Jewelry',
  'Novelty, Costumes & More',
  'Novelty',
  'Clothing',
  'Women',
  'Leggings'],
 ['Clothing, Shoes & Jewelry',
  'Women',
  'Clothing',
  'Lingerie, Sleep & Lounge',
  'Intimates',
  'Bras',
  'Everyday Bras'],
 ['Clothing, Shoes & Jewelry',
  'Women',
  'Petite',
  'Intimates',
  'Bras',
  'Everyday Bras']]